In [6]:
# -*- coding: utf-8 -*-
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.instruments as instruments

import json
import datetime
import pandas as pd
from dateutil.relativedelta import relativedelta
import numpy as np 
import matplotlib as plt
from IPython.display import display
import talib as ta

import time


#自分のアカウント、トークンをセット
accountID = "101-009-16415310-001"
access_token = '860c2d2dbbcd01b6d95939b6af6b5981-8664724ce0870220c55161d704137df6'

api = API(access_token=access_token, environment="practice")

# year_months =[ [2016, 1], [2016, 2], [2016, 3], [2016, 4], [2016, 5], [2016, 6], [2016, 7], [2016, 8], [2016, 9], [2016, 10], [2016, 11], [2016, 12],
#     [2017, 1], [2017, 2], [2017, 3], [2017, 4], [2017, 5], [2017, 6], [2017, 7], [2017, 8], [2017, 9], [2017, 10], [2017, 11], [2017, 12],
#     [2018, 1], [2018, 2], [2018, 3], [2018, 4], [2018, 5], [2018, 6], [2018, 7], [2018, 8], [2018, 9], [2018, 10], [2018, 11], [2018, 12],[2019, 1], [2019, 2], [2019, 3], [2019, 4], [2019, 5], [2019, 6], [2019, 7]]

## Oandaからcandleデータを取得する。 時差のため、現在のレート取得のため、-9h
def getCandleDataFromOanda(instrument, api, date_from, date_to, granularity):
    params = {
        "from": date_from.isoformat(),
        "to": date_to.isoformat(),
        "granularity": granularity,
    }
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    return api.request(r)

def oandaJsonToPythonList(JSONRes):
    
    #adjust = datetime.timedelta(hours=-4)
    adjust = datetime.timedelta(hours=9) #GMT to JPN
    
    data = []
    for res in JSONRes['candles']:
        temp = [datetime.datetime.fromisoformat(res['time'][:19]) + adjust] #indexとなる時間を追加
        
        temp.extend([datetime.datetime.fromisoformat(res['time'][:19]).hour]) #時間を追加
        
        month = datetime.datetime.fromisoformat(res['time'][:19]).month
        
        for i in range(1,13):
           
            if(month==i):
                temp.extend([1]) #月をonehotで追加
                #time.sleep(0.1)
            else:
                temp.extend([0])
                
        temp.extend([datetime.datetime.fromisoformat(res['time'][:19]).day])  #日を追加
        
        weekday = datetime.datetime.fromisoformat(res['time'][:19]).weekday
        
        for i in range(0,7):
            
            if(weekday==i):
                temp.extend([1]) #曜日をonehotで追加
                #time.sleep(0.00001)
            else:
                temp.extend([0])
        
        temp.extend([res['volume'],
            res['mid']['o'],
            res['mid']['h'],
            res['mid']['l'],
            res['mid']['c'],
            ])
        data.append(temp)
    return data



all_data = []
# year, monthでループ
#for year, month in year_months:

NY = datetime.timedelta(hours=-13) + datetime.timedelta(seconds=-1)#レートは時差を考慮して取得

# -13hしないと"This time is future"となることから、入力した時間はNY時間として認識される。


bias = datetime.timedelta(days=0)


date_from = datetime.datetime.now() + NY + datetime.timedelta(minutes=-50000) + bias
date_to =  datetime.datetime.now() + NY +bias

print("入力はNY(-13h)で行う。これにより最新のレートが取得できる。")
print("JPN: "+str(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
print("NY: FROM "+str(date_from.strftime('%Y-%m-%d %H:%M:%S'))+" TO "+str(date_to.strftime('%Y-%m-%d %H:%M:%S')))
#print("GMT(index): "+str(date_from+datetime.timedelta(hours=4))+" "+str(date_to+datetime.timedelta(hours=4)))

ret = getCandleDataFromOanda("USD_JPY", api, date_from, date_to, "M10")
twohours_data = oandaJsonToPythonList(ret)#取得したデータを格納



all_data.extend(twohours_data)#データを追加

#print(all_data)

#ここからall_dataに追加すればよい


# pandas DataFrameへ変換
now_df = pd.DataFrame(all_data)
now_df.columns = ['Datetime',"hour_range",
              "January","February","March","April","May","June","July","August","September","October","November","December",
              "day",
              "Sunday","Monday","Thuesday","Wednesday","Thursday","Friday","Saturday",
              'Volume', 'Open', 'High', 'Low', 'Close']
now_df = now_df.set_index('Datetime')

#print(now_df)

def adjust_now_data(df, num=1):
    
    df = df.astype(float)
    
    #以降全ての計算でレート終値を使う
    close = np.array(df["Close"])
    
    
    #特徴量を入れるための空のdataframeを作成
    df_feature = pd.DataFrame(columns=[ 
        "Close",
        "hour_range",
        #"January","February","March","April","May","June",
        #"July","August","September","October","November","December",
        "day",
        #"Sunday","Monday","Thuesday","Wednesday","Thursday","Friday","Saturday",
        "SMA_hour/current",
        "SMA_2hour/current",
        "RSI",
        "MACD",
        "BBANDS+2σ",
        "BBANDS-2σ",
        ])

    df_feature["Close"] = df["Close"]
    df_feature["hour_range"] = df["hour_range"]
    
#     df_feature["January"] = df["January"]
#     df_feature["February"] = df["February"]
#     df_feature["March"] = df["March"]
#     df_feature["April"] = df["April"]
#     df_feature["May"] = df["May"]
#     df_feature["June"] = df["June"]
#     df_feature["July"] = df["July"]
#     df_feature["August"] = df["August"]
#     df_feature["September"] = df["September"]
#     df_feature["October"] = df["October"]
#     df_feature["November"] = df["November"]
#     df_feature["December"] = df["December"]
    
    #print(df_feature.isnull().any())
    
    #print(df_feature.isnull().any())
    
#     days=["Sunday","Monday","Thuesday","Wednesday","Thursday","Friday","Saturday"]
#     for i in days:
#         df_feature[i] = df[i]

#     df_feature["Sunday"] = df["Sunday"]
#     df_feature["Monday"] = df["Monday"]
#     df_feature["Thuesday"] = df["Thuesday"]
#     df_feature["Wednesday"] = df["Wednesday"]
#     df_feature["Thursday"] = df["Thursday"]
#     df_feature["Friday"] = df["Friday"]
#     df_feature["Saturday"] = df["Saturday"]
    
    
    
    df_feature["day"] = df["day"]

    
    
    
    #以下、talibを用いてテクニカル指標（今回の学習で用いる特徴量）を算出しdf_feature入れる

    #単純移動平均は、単純移動平均値とその日の終値の比を特徴量として用いる
    df_feature["SMA_hour/current"]= ta.SMA(close, timeperiod=6) / close
    df_feature["SMA_2hour/current"]= ta.SMA(close, timeperiod=12) / close

    #RSI
    df_feature["RSI"] = ta.RSI(close, timeperiod=12)

    #MACD
    df_feature["MACD"], _ , _= ta.MACD(close, fastperiod=3, slowperiod=18, signalperiod=9)

    #ボリンジャーバンド 
    upper, middle, lower = ta.BBANDS(close, timeperiod=20, nbdevup=3, nbdevdn=3)
    df_feature["BBANDS+2σ"] = upper / close
    df_feature["BBANDS-2σ"] = lower / close

    
    c=[0,0,0]
    
    
    def classify(x):

        if x<0:
            c[0]+=1
            return 0
        elif 0<=x:
            c[1]+=1
            return 1
        else:
            c[2] +=1

    legs = 60
    
    classified = pd.DataFrame(columns=["class"])
    classified["class"] = (df["Close"].pct_change(int(legs/10))*100).apply(lambda x: classify(x))

    #classified = classified.shift(-1)
    classified = classified.shift(-int(legs/10))
    
    
    #df_adjusted=df_feature[30:len(df_feature)-2]
    #classified=classified[30:len(classified)-2]
    df_adjusted=df_feature[30:len(df_feature)-int(legs/10)]
    classified=classified[30:len(classified)-int(legs/10)]
    
    
    print("[down/up]")
    print(c)
    
    
    print(df_adjusted, classified)
    
    return df_adjusted, classified


from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
import pickle

def predict(now_df, filename = 'RF_01.sav'):
    
    day = 100
    
    data_num = 6*24*day #60分×24時間×days 間での予測精度を表示
    
    
    print("Recent 50minutes rate")
    print(now_df["Close"].tail())
    print("")
    
    
    X_now, y_now = adjust_now_data(now_df)

    print(X_now, y_now)

    loaded_model = pickle.load(open(filename, 'rb'))

    #result = loaded_model.score(X_now, y_now)

    print("The following is the accuracy during "+str(day)+"days.")
    
    from sklearn.metrics import accuracy_score
    
    print("accuracy_score: ", end="")
    print(accuracy_score(loaded_model.predict(X_now)[:data_num-6], y_now["class"].values[:data_num-6]))
    
    from sklearn.metrics import precision_score
    print("precision_score: ", end="")
    print(precision_score(loaded_model.predict(X_now)[:data_num-6], y_now["class"].values[:data_num-6], average=None))
    
    from sklearn.metrics import confusion_matrix
    print(confusion_matrix(y_now["class"].values[:data_num-6], loaded_model.predict(X_now)[:data_num-6]))
    
    return loaded_model.predict(X_now)

result = predict(now_df)



if result[-1] == 1:
    next_rate = "UP"
elif result[-1] == 0:
    next_rate = "DOWN"
print("next is ", end="")   
print(next_rate)



入力はNY(-13h)で行う。これにより最新のレートが取得できる。
JPN: 2020-10-07 10:00:07
NY: FROM 2020-09-02 03:40:06 TO 2020-10-06 21:00:06
Recent 50minutes rate
Datetime
2020-10-07 09:20:00    105.672
2020-10-07 09:30:00    105.661
2020-10-07 09:40:00    105.680
2020-10-07 09:50:00    105.658
2020-10-07 10:00:00    105.662
Name: Close, dtype: object

[down/up]
[1723, 1827, 6]
                       Close  hour_range  day  SMA_hour/current  \
Datetime                                                          
2020-09-02 21:40:00  106.170        12.0  2.0          1.000213   
2020-09-02 21:50:00  106.146        12.0  2.0          1.000242   
2020-09-02 22:00:00  106.188        13.0  2.0          0.999744   
2020-09-02 22:10:00  106.232        13.0  2.0          0.999498   
2020-09-02 22:20:00  106.244        13.0  2.0          0.999533   
...                      ...         ...  ...               ...   
2020-10-07 08:20:00  105.629        23.0  6.0          1.000022   
2020-10-07 08:30:00  105.625        23.0  6.0 